In [108]:
import numpy as np
import pandas as pd
#Use to create a scrollable dataframe that can display all rows instead of showing just the head rows.
#pd.set_option("display.max_rows", None)

In [2]:
df=pd.read_csv("glassdoor_jobs.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 15 columns):
Unnamed: 0           956 non-null int64
Job Title            956 non-null object
Salary Estimate      956 non-null object
Job Description      956 non-null object
Rating               956 non-null float64
Company Name         956 non-null object
Location             956 non-null object
Headquarters         956 non-null object
Size                 956 non-null object
Founded              956 non-null int64
Type of ownership    956 non-null object
Industry             956 non-null object
Sector               956 non-null object
Revenue              956 non-null object
Competitors          956 non-null object
dtypes: float64(1), int64(2), object(12)
memory usage: 112.2+ KB


### DATA CLEANING

In [4]:
'''DATA CLEANING'''
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [5]:
#Removing rows with -1 as salary
df=df[df['Salary Estimate']!='-1']
def removePerHour(val):
    if 'Per Hour' in val:
        return False
    else:
        return True
#Removed Datas with Salaries in PER HOUR
df=df[df['Salary Estimate'].apply(removePerHour)]
df

Unnamed: 0                                          Job Title  \
0             0                                     Data Scientist   
1             1                          Healthcare Data Scientist   
2             2                                     Data Scientist   
3             3                                     Data Scientist   
4             4                                     Data Scientist   
5             5                                     Data Scientist   
6             6                                     Data Scientist   
7             7                                     Data Scientist   
8             8                                 Research Scientist   
9             9                                     Data Scientist   
10           10                                     Data Scientist   
11           11                                     Data Scientist   
12           12                  Staff Data Scientist - Technology   
13           13                                       Data Analyst   
14           14                                     Data Scientist   
15           15                                    Data Engineer I   
16           16                            Scientist I/II, Biology   
17           17                            Customer Data Scientist   
18           18             Data Scientist - Health Data Analytics   
19           19                                     Data Scientist   
20           20                                     Data Scientist   
21           21           Senior Data Scientist / Machine Learning   
22           22                      Data Scientist - Quantitative   
23           23                                     Data Scientist   
25           25                                     Data Scientist   
26           26                      Digital Health Data Scientist   
27           27                                     Data Scientist   
28           28                             Associate Data Analyst   
29           29                            Clinical Data Scientist   
30           30                                     Data Scientist   
31           31                                     Data Scientist   
32           32                                     Data Scientist   
33           33           Data Scientist / Machine Learning Expert   
34           34                                     Data Scientist   
35           35                                     Data Scientist   
36           36                                   Web Data Analyst   
37           37                                     Data Scientist   
38           38                                     Data Scientist   
39           39                              Senior Data Scientist   
40           40                                      Data Engineer   
41           41                                       Data Analyst   
43           43                                      Data Engineer   
44           44            Data Scientist - Algorithms & Inference   
45           45                                          Scientist   
46           46                              Senior Data Scientist   
48           48                                     Data Scientist   
49           49                                Lead Data Scientist   
50           50                             Associate Data Analyst   
51           51                        Spectral Scientist/Engineer   
52           52  College Hire - Data Scientist - Open to Decemb...   
54           54                                     Data Scientist   
55           55             Data Scientist, Office of Data Science   
56           56                               Data Science Analyst   
57           57                                     Data Scientist   
58           58                                     Data Scientist   
59           59                                     Data Scientist   
60           60

## SALARY PARSING-
### OBTAINED MIN, MAX AND AVG SALARIES

In [6]:
#Salary Parsing
def remove_para(value):
    return value.split('(')[0]
#REMOVED THE PARANTHESIS
salary=df['Salary Estimate'].apply(remove_para)
#REMOVED THE 'K' AND '$'
def remove_k_dol(val):
    val=val.replace('K', ' ').replace('-',' ')
    return val
salary_mod=salary.apply(remove_k_dol)
#Removed the 'Employer provided salary section and trimmed from both end'
def reformat_sal(val):
    if ':' in val:
        val= val.split(':')[1]
    return val
def trim(val):
    return val.strip()
salary_mod2=salary_mod.apply(reformat_sal).apply(trim)
salary_mod2[1]


'$63  $112'

In [18]:
def split_getmin(s):
    return int(s.split('  ')[0].split('$')[1])
df['min sal']=salary_mod2.apply(split_getmin)

In [26]:
df['min sal']
df['min sal'].dtype

dtype('int64')

In [22]:
def split_getmax(s):
    return int(s.split('  ')[1].split('$')[1])
df['max sal']=salary_mod2.apply(split_getmax)

In [25]:
df['max sal']
df['max sal'].dtype

dtype('int64')

In [30]:
df['avg']=(df['min sal']+df['max sal'])/2
df['avg'].dtype

dtype('float64')

### Cleaned the company names

In [45]:
df['Company Name']=df['Company Name'].apply(lambda x: x.split('\n')[0])

### Cleaning State and HQ fields

In [87]:
df['State']=df['Location'].apply(lambda x: x.split(',')[1])

#Check if HQ and Job are in same state
df['same state']=df.apply(lambda x: 1 if x['Location']== x['Headquarters'] else 0, axis=1)
#Fixed state of Los Angeles to LA for uniformity
df['State']=df['State'].apply(lambda x: 'LA' if x==' Los Angeles' else x)

### Company Age

In [88]:
#Age of company
df['age']=df['Founded'].apply(lambda x: x if x==-1 else 2020-x)

### Parsing of job description

### Python Keyword

In [93]:
df['python_reqd']=df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
df['python_reqd'].value_counts()

1    392
0    326
Name: python_reqd, dtype: int64

In [94]:
df['spark_reqd']=df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
df['spark_reqd'].value_counts()

0    551
1    167
Name: spark_reqd, dtype: int64

In [98]:
df['aws_reqd']=df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() or 'amazon web services' in x.lower() else 0)
df['aws_reqd'].value_counts()

0    543
1    175
Name: aws_reqd, dtype: int64

In [99]:
df['excel_reqd']=df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df['excel_reqd'].value_counts()

1    382
0    336
Name: excel_reqd, dtype: int64

In [101]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'min sal', 'max sal', 'avg', 'State', 'same state', 'age',
       'python_reqd', 'spark_reqd', 'aws_reqd', 'excel_reqd'],
      dtype='object')

In [103]:
df=df.drop(['Unnamed: 0'], axis=1)

In [106]:
df.to_csv('salary_data_cleaned.csv', index=False)